## **Chapter 5: Data Pre-processing**
Ex2: AA data
Cho dữ liệu về thông tin các chuyến bay trong thư mục
"AA_data"
Yêu cầu:
1. Đọc dữ liệu => df
2. Cho biết dữ liệu có bao nhiêu dòng, in scheme. Hiển thị 5 dòng dữ liệu đầu tiên.
3. Kiểm tra dữ liệu NaN, null
4. Kiểm tra dữ liệu trùng. Xóa dữ liệu trùng.
5. Trong df, thêm cột 'airport' lấy dữ liệu từ cột 'Destination Airport', định dạng chữ thường cho
nội dung
6. Trong df, thêm cột 'date' lấy dữ liệu từ cột 'Date (MM/DD/YYYY)', sau đó xóa bỏ cột 'Date
(MM/DD/YYYY)'
7. Trong df, đổi tên cột "Flight Number" thành "flight_num", cột "Actual elapsed time (Minutes)"
thành "actual_time"
8. Lưu df dưới dạng Parquet format với tên là "AA_DFW_ALL.parquet"
9. Đọc parquet "AA_DFW_ALL.parquet" => df_new
10. Tạo một bảng tạm 'flights'. Cho biết trung bình của 'actual_time' trong 'flight'
11. Caching các dòng dữ liệu duy nhất của df_new. Đếm số dòng. Cho biết thời gian thực hiện
các công việc này.
12. Đếm lại số dòng. Cho biết thời gian thực hiện các công việc này.
13. Kiểm tra xem df_new có trong cache hay không? Nếu có thì bỏ df_new ra khỏi cache.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

spark-2.4.0-bin-hadoop2.7/
spark-2.4.0-bin-hadoop2.7/python/
spark-2.4.0-bin-hadoop2.7/python/setup.cfg
spark-2.4.0-bin-hadoop2.7/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/resultiterable.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/heapq3.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/join.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/version.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/rdd.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/java_gateway.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/find_spark_home.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/_globals.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/worker.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/feature.py
spark-2.4.0-bin-hadoop2.7/python/pyspark

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
%cd '/content/gdrive/My Drive/LDS9_K273_ONLINE_Đinh Viết Trung'

/content/gdrive/My Drive/LDS9_K273_ONLINE_Đinh Viết Trung


In [4]:
# import libraries
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from pyspark.sql.functions import mean, stddev, col, log
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql import types 
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import coalesce, first, lit
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import datediff
from pyspark.sql.functions import when

In [5]:
from pyspark import SparkContext
sc = SparkContext()

In [6]:
spark = SparkSession(sc)

In [7]:
#1. 
df = spark.read.csv(["data/AA_data"], header=True, inferSchema=True)

In [8]:
#2.
df.count()

444360

In [9]:
df.show(5)

+-----------------+-------------+-------------------+-----------------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)|
+-----------------+-------------+-------------------+-----------------------------+
|       01/01/2014|            5|                HNL|                          519|
|       01/01/2014|            7|                OGG|                          505|
|       01/01/2014|           35|                SLC|                          174|
|       01/01/2014|           43|                DTW|                          153|
|       01/01/2014|           52|                PIT|                          137|
+-----------------+-------------+-------------------+-----------------------------+
only showing top 5 rows



In [10]:
df.printSchema()

root
 |-- Date (MM/DD/YYYY): string (nullable = true)
 |-- Flight Number: integer (nullable = true)
 |-- Destination Airport: string (nullable = true)
 |-- Actual elapsed time (Minutes): integer (nullable = true)



In [11]:
from pyspark.sql.functions import col, udf
from pyspark.sql.functions import isnan, when, count, col

In [12]:
#3. Kiểm tra dữ liệu NaN, null
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).toPandas().T

,0
Date (MM/DD/YYYY),0
Flight Number,0
Destination Airport,0
Actual elapsed time (Minutes),0


In [13]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in
 df.columns]).toPandas().T


,0
Date (MM/DD/YYYY),0
Flight Number,0
Destination Airport,0
Actual elapsed time (Minutes),0


In [14]:
#4. Kiểm tra dữ liệu trùng. Xóa dữ liệu trùng.
num_rows = df.count()
num_dist_rows = df.distinct().count()
dup_rows = num_rows - num_dist_rows
dup_rows

0

**Lazy processing operations**

In [15]:
from pyspark.sql.functions import *

In [16]:
#5. Add the airport column using the F.lower() method
df = df.withColumn('airport', lower(df['Destination Airport']))


In [17]:
df = df.drop('Destination Airport')

In [18]:
df.show(5)

+-----------------+-------------+-----------------------------+-------+
|Date (MM/DD/YYYY)|Flight Number|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-----------------------------+-------+
|       01/01/2014|            5|                          519|    hnl|
|       01/01/2014|            7|                          505|    ogg|
|       01/01/2014|           35|                          174|    slc|
|       01/01/2014|           43|                          153|    dtw|
|       01/01/2014|           52|                          137|    pit|
+-----------------+-------------+-----------------------------+-------+
only showing top 5 rows



In [19]:
#6. Add column date, using column Date (MM/DD/YYYY), drop Date (MM/DD/YYYY)
df = df.withColumn('date', df['Date (MM/DD/YYYY)'])
df = df.drop('Date (MM/DD/YYYY)')

In [20]:
df.show(5)


+-------------+-----------------------------+-------+----------+
|Flight Number|Actual elapsed time (Minutes)|airport|      date|
+-------------+-----------------------------+-------+----------+
|            5|                          519|    hnl|01/01/2014|
|            7|                          505|    ogg|01/01/2014|
|           35|                          174|    slc|01/01/2014|
|           43|                          153|    dtw|01/01/2014|
|           52|                          137|    pit|01/01/2014|
+-------------+-----------------------------+-------+----------+
only showing top 5 rows



In [22]:
#7.
df = df.withColumnRenamed("Flight Number", "flight_num")
df = df.withColumnRenamed("Actual elapsed time (Minutes)", "actual_time")

In [23]:
df.show(5)

+----------+-----------+-------+----------+
|flight_num|actual_time|airport|      date|
+----------+-----------+-------+----------+
|         5|        519|    hnl|01/01/2014|
|         7|        505|    ogg|01/01/2014|
|        35|        174|    slc|01/01/2014|
|        43|        153|    dtw|01/01/2014|
|        52|        137|    pit|01/01/2014|
+----------+-----------+-------+----------+
only showing top 5 rows



In [24]:
# 8. Save the df DataFrame in Parquet format
df.write.parquet('AA_DFW_ALL.parquet.1', mode='overwrite')


In [25]:
# 9 Read the Parquet file into a new DataFrame 
df_new = spark.read.parquet('AA_DFW_ALL.parquet.1')


**SQL and Parquet**

In [26]:
# 10. Register the temp table
df_new.createOrReplaceTempView('flights')

In [27]:
# Run a SQL query of the average Actual elapsed time
avg_duration = spark.sql('SELECT avg(actual_time) from flights').collect()[0]
print('The average flight time is: %d' % avg_duration)

The average flight time is: 146


### Improving Performance
- Caching a DataFrame
    - Caching can improve performance when reusing DataFrames

In [30]:
import time

In [31]:
# 11.
start_time = time.time()
# Add caching to the unique rows in df_new
df_new = df_new.distinct().cache()
# Count the unique rows in df_new, noting how long the operation takes
print("Counting %d rows took %f seconds" %
 (df_new.count(), time.time() - start_time))

Counting 444360 rows took 7.169266 seconds


In [33]:
# Count the rows again, noting the variance in time of a cached DataFrame
start_time = time.time()
print("Counting %d rows again took %f seconds" % (df_new.count(), time.time() - start_time))

Counting 444360 rows again took 1.135239 seconds


In [34]:
# Determine if df_new is in the cache
print("Is df_new cached?: %s" % df_new.is_cached)
print("Removing df_new from cache")
# Remove df_new from the cache
df_new.unpersist()
# Check the cache status again
print("Is df_new cached?: %s" % df_new.is_cached)

Is df_new cached?: True
Removing df_new from cache
Is df_new cached?: False


In [35]:
# Name of the Spark application instance
app_name = spark.conf.get('spark.app.name')
# Driver TCP port
driver_tcp_port = spark.conf.get('spark.driver.port')
# Number of join partitions
num_partitions = spark.conf.get('spark.sql.shuffle.partitions')
# Show the results
print("Name: %s" % app_name)
print("Driver TCP port: %s" % driver_tcp_port)
print("Number of partitions: %s" % num_partitions

SyntaxError: ignored

In [40]:
#Store the number of partitions in variable
before = df_new.rdd.getNumPartitions()

# Configure Spark to use 500 partitions
spark.conf.set('spark.sql.shuffle.partitions', 500)

# Recreate the DataFrame using the departures data file
df_new = spark.read.parquet('data/AA_data/AA_DFW_ALL.parquet').distinct()

# Print the number of partitions for each instance
print("Partition count before change: %d" % before)
print("Partition count after change: %d" % df_new.rdd.getNumPartitions())


Partition count before change: 200
Partition count after change: 500


In [ ]:
### save data to json file
#df_new.write.json('AA_DFW_ALL.json')